# Energy evaluation

Let's import protlego

In [1]:
import sys
from protlego import *

Let's fetch the same example we have been working on, the relationship between a TIM-barrel and a P-loop domain:

In [2]:
myhit= fetch_id('10002347')

Build all possible chimera-combinations from query and subject. As explained in detail in the "Building chimeras" section. The variable chimeras stores all the possible combinations,

In [5]:
a=Builder(myhit)
aln=a.get_alignment(myhit.query,myhit.no)
qPDB, sPDB = a.superimpose_structures(aln,partial_alignment=True)
chimeras=a.build_chimeras(partial_alignment=True)

2020-07-31 10:42:24,415 - protlego - INFO - Attempting to download domain d1thfd_ from the SCOP server
2020-07-31 10:42:25,965 - protlego - INFO - File downloaded as /tmp/tmpq3n80mub.pdb
2020-07-31 10:42:25,967 - protlego - INFO - Attempting to download domain d2yhsa2 from the SCOP server
2020-07-31 10:42:27,176 - protlego - INFO - File downloaded as /tmp/tmpilhmfsc5.pdb
2020-07-31 10:42:27,178 - protlego - INFO - Loading /tmp/tmpq3n80mub.pdb as a chimera object
2020-07-31 10:42:27,269 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2020-07-31 10:42:27,271 - protlego - INFO - Loading /tmp/tmpilhmfsc5.pdb as a chimera object
2020-07-31 10:42:27,346 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2020-07-31 10:42:27,347 - protlego - I

### 1. Energy calculation using molecular mechanics (Amber and Charmm)

We can evaluate the potential energy of a chimera with the Charmm and Amber forcefieldds. As an option we can keep intermediate file in a folder or not. The function score_pot_energy takes a Chimera object as input and applies the selected forcefield:

In [6]:
chimera=Builder.mutate_nonstandards(chimeras['comb2_190'])
_,_=score_pot_energy(chimeras['comb2_190'], 'amber')

2020-07-31 10:43:09,212 - propka - INFO - No pdbfile provided
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


Potential Energy: 195233854730.05392 kcal/mol


The built chimeras sometimes present internal clashes. For this reason, they tend to present very high potential energy scores. To compute a more realistic measure, we can minimize the structure using the minimize_potential_energy function. There's the possibility to run the minimization with GPU acceleration, and one can also choose to restraint the backbone or not.

In [7]:
minimize_potential_energy(chimera, 'amber', cuda=True, restraint_backbone=False)

/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'TM3P'
  resname, ParameterWarning)
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'MGU1'
  resname, ParameterWarning)
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'MGU2'
  resname, ParameterWarning)
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:804: UserWarning: No charge for WEI3
  warnings.warn('No charge for %s' % resname)


Energy before minimization 4187401915.1074705 kcal/mol
Energy after minimization -3985.5893887369366 kcal/mol


(Quantity(value=-3985.5893887369366, unit=kilocalorie/mole),
 Protein with:
 Number of residues: 239
 Number of atoms: 3673)

We can do the same with the Charmm forcefield:

In [8]:
minimize_potential_energy(chimera, 'charmm', cuda=True, restraint_backbone=False)

/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'TM3P'
  resname, ParameterWarning)
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'MGU1'
  resname, ParameterWarning)
/sw/sci/app/protlego/miniconda3/envs/mod_protlego/lib/python3.6/site-packages/parmed/charmm/parameters.py:797: ParameterWarning: Replacing residue 'MGU2'
  resname, ParameterWarning)


Energy before minimization 4187401915.1070247 kcal/mol
Energy after minimization -4033.609105974216 kcal/mol


(Quantity(value=-4033.609105974216, unit=kilocalorie/mole),
 Protein with:
 Number of residues: 239
 Number of atoms: 3673)